# Preprocess data

In [18]:
import json

def load_and_convert_data(input_file):
    vehicles_data = {}

    with open(input_file, 'r') as f:
        for line in f:
            obj = json.loads(line.strip())
            vehicle_id = obj['vehicle_id']
            timestamps = [ts / 1000 for ts in obj['timestamp']]  # 毫秒转秒
            local_ys = [y * 0.3048 for y in obj['local_y']]  # 英尺转米
            v_vel = [vel * 0.681818 for vel in obj['v_vel']]  # 英尺/秒转英里/小时
            v_acc = obj['v_acc']
            lane_ids = obj['lane_id']
            followings = obj['following']
            space_headways = obj['space_headway']

            vehicles_data[vehicle_id] = {
                'timestamp': timestamps,
                'local_y': local_ys,
                'v_vel': v_vel,
                'v_acc': v_acc,
                'lane_id': lane_ids,
                'following': followings,
                'space_headway': space_headways,
            }

    return vehicles_data

input_file = '../trajectories-0400-0415.json'
vehicles_data = load_and_convert_data(input_file)
print("Load complete!")

Load complete!


# Find lane-change event meeting conditions

In [19]:
def detect_lane_changes(vehicles_data, output_file):
    lane_changes = []
    
    for vehicle_id, data in vehicles_data.items():
        lane_id = data['lane_id']
        timestamps = data['timestamp']
        following = data['following']

        for i in range(1, len(lane_id)):
            if lane_id[i] != lane_id[i - 1] and following[i] != 0:
                lane_changes.append({
                    'vehicle_id': vehicle_id,
                    'before_lane_id': lane_id[i-1],
                    'after_lane_id': lane_id[i],
                    'before_change_timestamp': timestamps[i - 1],
                    'after_change_timestamp': timestamps[i],
                    'after_following_id': following[i]
                })

    with open(output_file, 'w') as f:
        for change in lane_changes:
            json.dump(change, f)
            f.write('\n')

output_file = 'trajectories-0400-0415-pre-after-following.json'
detect_lane_changes(vehicles_data, output_file)

print('Record complete!')

Record complete!


# Complete the detailed file

In [20]:
def extract_detailed_after_following(vehicles_data, lane_changes_file, output_file, delta_t=30):
    detailed_after_following = []
    count = 0

    with open(lane_changes_file, 'r') as f:
        for line in f:
            change = json.loads(line.strip())
            after_following_id = change['after_following_id']
            before_change = change['before_change_timestamp']
            after_change = change['after_change_timestamp']

            if after_following_id in vehicles_data:
                after_following_data = vehicles_data[after_following_id]
                timestamps = after_following_data['timestamp']
                space_headways = after_following_data['space_headway']
                v_accs = after_following_data['v_acc']
                v_vels = after_following_data['v_vel']

                start_time = before_change - 5
                end_time = after_change + delta_t / 2

                event_details = {
                    'vehicle_id': change['vehicle_id'],
                    'before_change_timestamp': before_change,
                    'after_change_timestamp': after_change,
                    'timestamps': [],
                    'gaps': [],
                    'accs': [],
                    'vels': []
                }

                for i, ts in enumerate(timestamps):
                    if start_time <= ts <= end_time:
                        event_details['timestamps'].append(ts)
                        event_details['gaps'].append(space_headways[i])
                        event_details['accs'].append(v_accs[i])
                        event_details['vels'].append(v_vels[i])

                if event_details['timestamps']:
                    detailed_after_following.append(event_details)

                count += 1
                if count % 200 == 0:
                    print(f"Processed {count} vehicles")

    with open(output_file, 'w') as f:
        for record in detailed_after_following:
            json.dump(record, f)
            f.write('\n')

# 生成详细的变道后后车信息
detailed_output_file = 'trajectories-0400-0415-detailed-after-following.json'
extract_detailed_after_following(vehicles_data, output_file, detailed_output_file)

print('Detail record complete!')

Processed 200 vehicles
Processed 400 vehicles
Processed 600 vehicles
Processed 800 vehicles
Detail record complete!
